In [1]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import JointState
from std_msgs.msg import String
import time

In [2]:
joint_state = JointState()

class ARCSnakeListener(Node):
    '''
        Gets joint angle information from the snake
    '''
    def __init__(self):
        super().__init__('arcsnake_listener')
        self.subscription = self.create_subscription(
            JointState,
            '/arcsnake/joints_current',
            self.listener_callback,
            10)
        self.subscription  # prevent unused variable warning
        self.joint_publisher = self.create_publisher(JointState, '/arcsnake/joints_desired', 10)
        self.state_publisher = self.create_publisher(String, '/arcsnake/state', 10)

    def listener_callback(self, msg):
        global joint_state
        joint_state = msg

In [3]:
rclpy.init()

arcsnake_listener = ARCSnakeListener()
rclpy.spin_once(arcsnake_listener)

print(joint_state)


sensor_msgs.msg.JointState(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id=''), name=['seg0_screw', 'seg0_uJoint2', 'seg1_uJoint1', 'seg1_screw', 'seg1_uJoint2', 'seg2_uJoint1', 'seg2_screw', 'seg2_uJoint2', 'seg3_uJoint1', 'seg3_screw'], position=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], velocity=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], effort=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])


In [4]:
arcsnake_listener.state_publisher.publish(String(data="start"))

In [7]:
rclpy.spin_once(arcsnake_listener)
print(joint_state.name)
print(joint_state.position)
print(joint_state.velocity)

['seg0_screw', 'seg0_uJoint2', 'seg1_uJoint1', 'seg1_screw', 'seg1_uJoint2', 'seg2_uJoint1', 'seg2_screw', 'seg2_uJoint2', 'seg3_uJoint1', 'seg3_screw']
array('d', [-2.703340478414017, 0.02548180707911721, -0.054724005364804026, -1.2470377505499486, -0.034033920413889425, 0.034827251892068666, 2.5542893602937013, 0.20515552025715167, 0.0, 2.075371013546457])
array('d', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])


In [8]:
# Set torques to 0
joint_state.effort = [0.0]*len(joint_state.effort)
arcsnake_listener.joint_publisher.publish(joint_state)

In [13]:
arcsnake_listener.state_publisher.publish(String(data="stop"))

In [13]:
# Wheeling motion
speed = 15

joint_state.velocity[0] = -speed
joint_state.velocity[3] = speed
joint_state.velocity[6] = speed
joint_state.velocity[9] = speed

arcsnake_listener.joint_publisher.publish(joint_state)

In [13]:
rclpy.spin_once(arcsnake_listener)
print(joint_state.position[-2])

-0.03086059450117246


In [20]:
# From the current joint state, move -2 joint to pi/2 with linear interpolation and time.sleep(1 s)
import numpy as np
import time

motor_idx = 2
offset_goal =  0.7
# offset_goal = np.pi/2

pos_traj = np.linspace(joint_state.position[motor_idx], joint_state.position[motor_idx] + offset_goal, 500)
for pos in pos_traj:
    joint_state.position[motor_idx] = pos
    # joint_state.velocity[-1] = 0
    arcsnake_listener.joint_publisher.publish(joint_state)
    time.sleep(0.01)

In [9]:
rclpy.spin_once(arcsnake_listener)
print(joint_state.position)

array('d', [-2.703340478414017, 0.02548180707911721, -0.054724005364804026, -1.248434013951544, -0.034033920413889425, 0.034827251892068666, 2.552893096892106, 0.20515552025715167, 0.0, 1.9949113350295187])


In [14]:
#Screwing motion

# Keep all positions constant
# Set all toqrues initially to 0
joint_state.effort = [0.0]*len(joint_state.effort)

torque_value = -2.5
# positive torque for motor_idx 1
joint_state.effort[1] = torque_value

# positive torque for motor_idx -3
joint_state.effort[-3] = -torque_value

# Screwing motion
speed = 15

joint_state.velocity[0] = -speed
joint_state.velocity[3] = speed
joint_state.velocity[6] = speed
joint_state.velocity[9] = speed

# Publish the joint state
arcsnake_listener.joint_publisher.publish(joint_state)

In [17]:
print(joint_state.position)
print(joint_state.velocity)
print(joint_state.effort)

array('d', [0.21886428820008894, 0.02924219828568681, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
array('d', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
array('d', [0.0, -2.5, 0.0, 0.0, 0.0, 0.0, 0.0, 2.5, 0.0, 0.0])


In [22]:
rclpy.spin_once(arcsnake_listener)

In [10]:
# wheeling motion
speed = 0

joint_state.velocity[0] = -speed
joint_state.velocity[3] = speed
joint_state.velocity[6] = -speed
joint_state.velocity[9] = speed


arcsnake_listener.joint_publisher.publish(joint_state)

In [14]:
factor = 8
number_of_steps = 200
for i in range(number_of_steps):
    joint_state.velocity[0] = (factor*5)*i/number_of_steps  + 2
    joint_state.velocity[6] = -(factor*5)*i/number_of_steps  + 2
    
    arcsnake_listener.joint_publisher.publish(joint_state)
    time.sleep(0.01)


In [11]:
change_in_radians = 0.1
number_of_steps = 200
for i in range(number_of_steps):
    joint_state.position[2] += change_in_radians/number_of_steps
    joint_state.position[5] += -change_in_radians/number_of_steps
    arcsnake_listener.joint_publisher.publish(joint_state)
    time.sleep(0.01)

In [19]:
change_in_radians = 0.1
number_of_steps = 200
for i in range(number_of_steps):
    joint_state.position[2] += -change_in_radians/number_of_steps
    arcsnake_listener.joint_publisher.publish(joint_state)
    time.sleep(0.01)

In [20]:
change_in_radians = 0.1
number_of_steps = 200
for i in range(number_of_steps):
    joint_state.position[5] +=  change_in_radians/number_of_steps
    arcsnake_listener.joint_publisher.publish(joint_state)
    time.sleep(0.01)

In [8]:
import math

In [9]:
start_point_2 = joint_state.position[2]
start_point_5 = joint_state.position[5]

In [15]:
max_angle = 0.5

for t in range(10000):
    f = 0.01
    joint_state.position[2] = start_point_2 + max_angle*math.sin(f*t)
    joint_state.position[5] = start_point_5 + max_angle*math.sin(f*t)
    arcsnake_listener.joint_publisher.publish(joint_state)
    time.sleep(0.01)

KeyboardInterrupt: 

In [16]:
arcsnake_listener.state_publisher.publish(String(data="off"))

In [17]:
# Destroy the node
arcsnake_listener.destroy_node()
rclpy.shutdown()